# ReAct 模式实验：Reasoning + Acting

## 📚 什么是 ReAct？

**ReAct (Reasoning + Acting)** = 推理 + 行动，让模型在思考和执行工具调用之间交替。

### 核心流程

```
Thought（思考）→ Action（行动）→ Observation（观察）→ [循环] → Answer（答案）
```

### vs CoT

| 维度 | CoT | ReAct |
|-----|-----|-------|
| 能力 | 仅推理 | 推理+工具调用 |
| 外部信息 | ❌ | ✅ |
| 主要失败 | 推理错误 | Action Loop、工具幻觉 |

## 🎯 本实验内容

1. **成功示例**：正常的 ReAct 流程
2. **失败模式1**：Action Loop（无限循环）
3. **失败模式2**：Thought 碎片化
4. **失败模式3**：工具幻觉
5. **失败模式4**：过早终止

## 🤖 测试模型

使用 **Qwen3 8B** 模型

In [ ]:
# === 环境准备 ===
import subprocess
import time
import requests
import json
import re

# 安装并启动 Ollama
print("⬇️ 正在安装 Ollama...")
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=False)
print("✅ Ollama 安装完成")

print("🚀 正在后台启动 Ollama 服务...")
ollama_server = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✅ Ollama 服务已运行")

# 下载模型
print("⬇️ 正在下载 qwen3:8b...")
subprocess.run(["ollama", "pull", "qwen3:8b"], check=False)
print("✅ 模型下载完成")

In [ ]:
# === 模拟工具函数 ===

# 模拟数据库
KNOWLEDGE_BASE = {
    "北京天气": "北京今天晴天，温度15-25度",
    "上海天气": "上海今天多云，温度18-28度",
    "Python": "Python是一种高级编程语言，由Guido van Rossum于1991年创建",
    "机器学习": "机器学习是人工智能的一个分支，让计算机从数据中学习",
    "埃菲尔铁塔": "埃菲尔铁塔位于法国巴黎，高324米，建于1889年",
    "巴黎": "巴黎是法国首都，人口约220万",
}

def tool_search(query):
    """搜索工具：查询知识库"""
    for key in KNOWLEDGE_BASE:
        if query in key or key in query:
            return KNOWLEDGE_BASE[key]
    return f"未找到关于'{query}'的信息"

def tool_calculator(expression):
    """计算器工具：执行数学计算"""
    try:
        # 安全的计算（仅允许基本运算）
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"计算错误: {str(e)}"

def tool_lookup(keyword):
    """查找工具：在上一次搜索结果中查找关键词"""
    # 这里简化实现，实际应该从历史结果中查找
    if keyword in str(KNOWLEDGE_BASE):
        return f"找到关键词'{keyword}'"
    return f"未找到关键词'{keyword}'"

# 工具注册表
TOOLS = {
    "Search": tool_search,
    "Calculator": tool_calculator,
    "Lookup": tool_lookup,
}

print("✅ 工具函数加载完成")
print(f"可用工具: {list(TOOLS.keys())}")

In [ ]:
# === ReAct 执行引擎 ===

def react_executor(question, max_iterations=5, verbose=True):
    """
    ReAct 执行引擎
    
    参数:
        question: 用户问题
        max_iterations: 最大迭代次数（防止无限循环）
        verbose: 是否打印详细过程
    
    返回:
        (final_answer, history, success_flag)
    """
    # 系统提示：定义 ReAct 格式
    system_prompt = """你是一个问题解决助手。你可以使用以下工具：

工具列表：
1. Search[query] - 搜索知识库获取信息
2. Calculator[expression] - 执行数学计算
3. Lookup[keyword] - 在之前的搜索结果中查找关键词

请严格按照以下格式回答：

Thought: [你的思考过程]
Action: [工具名称]
Action Input: [工具输入]

你会收到工具执行结果：
Observation: [工具返回结果]

然后继续思考和行动，直到得出最终答案：

Thought: 我现在知道最终答案了
Final Answer: [你的最终答案]

重要：每次只输出一个 Thought-Action-Action Input 组合！"""

    history = []
    context = f"Question: {question}\n"
    
    for iteration in range(max_iterations):
        if verbose:
            print(f"\n{'='*60}\n第 {iteration + 1} 轮迭代\n{'='*60}")
        
        # 调用模型
        full_prompt = system_prompt + "\n\n" + context
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": "qwen3:8b", "prompt": full_prompt, "stream": False}
        )
        
        if response.status_code != 200:
            return None, history, False
        
        model_output = response.json()["response"].strip()
        
        if verbose:
            print(f"📤 模型输出:\n{model_output}\n")
        
        history.append({"iteration": iteration + 1, "output": model_output})
        
        # 解析模型输出
        # 1. 检查是否包含 Final Answer
        if "Final Answer:" in model_output:
            match = re.search(r"Final Answer:\s*(.+)", model_output, re.DOTALL)
            if match:
                final_answer = match.group(1).strip()
                if verbose:
                    print(f"✅ 找到最终答案: {final_answer}")
                return final_answer, history, True
        
        # 2. 提取 Thought
        thought_match = re.search(r"Thought:\s*(.+?)(?=Action:|$)", model_output, re.DOTALL)
        thought = thought_match.group(1).strip() if thought_match else "未找到思考"
        
        # 3. 提取 Action
        action_match = re.search(r"Action:\s*(\w+)", model_output)
        if not action_match:
            if verbose:
                print("⚠️ 未找到 Action，终止")
            return None, history, False
        
        action = action_match.group(1)
        
        # 4. 提取 Action Input
        action_input_match = re.search(r"Action Input:\s*(.+?)(?=\n|$)", model_output, re.DOTALL)
        action_input = action_input_match.group(1).strip() if action_input_match else ""
        
        if verbose:
            print(f"💭 Thought: {thought}")
            print(f"🔧 Action: {action}")
            print(f"📝 Action Input: {action_input}")
        
        # 5. 执行工具
        if action in TOOLS:
            observation = TOOLS[action](action_input)
            if verbose:
                print(f"👁️ Observation: {observation}")
        else:
            observation = f"错误：未知工具 '{action}'。可用工具: {list(TOOLS.keys())}"
            if verbose:
                print(f"❌ {observation}")
        
        # 6. 将 Observation 添加到上下文
        context += f"\n{model_output}\nObservation: {observation}\n"
    
    # 达到最大迭代次数
    if verbose:
        print(f"\n⚠️ 达到最大迭代次数 {max_iterations}，未找到答案")
    return None, history, False

print("✅ ReAct 执行引擎加载完成")

## ✅ 实验 1：成功的 ReAct 流程

**测试问题**：需要多步推理和工具调用的复杂问题

**预期流程**：
1. **Thought** → 分析问题，决定使用工具
2. **Action** → 调用工具
3. **Observation** → 获取结果
4. **Thought** → 基于结果继续推理
5. **循环** → 直到得出 Final Answer

**成功标志**：
- 清晰的思考链条
- 合理的工具选择
- 正确的最终答案

In [ ]:
# 实验 1：成功案例
question = "埃菲尔铁塔比上海东方明珠（468米）高多少米？"

print("🔍 问题:", question)
print("\n" + "="*60)

answer, history, success = react_executor(question, max_iterations=5, verbose=True)

print("\n" + "="*60)
print("📊 实验结果")
print("="*60)
print(f"✅ 成功: {success}")
print(f"🔢 迭代次数: {len(history)}")
if answer:
    print(f"💡 最终答案: {answer}")
else:
    print("❌ 未能得出答案")

# 分析流程
print("\n📈 流程分析:")
for i, step in enumerate(history, 1):
    print(f"\n第 {i} 步:")
    output = step['output']
    has_thought = "Thought:" in output
    has_action = "Action:" in output
    has_final = "Final Answer:" in output
    print(f"  - Thought: {'✅' if has_thought else '❌'}")
    print(f"  - Action: {'✅' if has_action else '❌'}")
    print(f"  - Final Answer: {'✅' if has_final else '❌'}")

## ❌ 实验 2：失败模式 1 - Action Loop（行动循环）

**问题描述**：模型陷入重复调用相同工具的循环，无法推进

**典型表现**：
```
Thought: 我需要搜索
Action: Search
Observation: [结果]

Thought: 我需要再次搜索  ← 重复！
Action: Search
Observation: [相同或类似结果]
...无限循环
```

**触发原因**：
1. 问题超出工具能力范围
2. 模型没有正确理解 Observation
3. 缺少"停止"的判断逻辑

**检测方法**：
- 设置 `max_iterations` 限制
- 检测连续相同的 Action 调用

In [ ]:
# 实验 2：Action Loop 失败案例
# 问一个知识库中不存在的问题，容易触发重复搜索
question_loop = "今天深圳的天气怎么样？"

print("🔍 问题:", question_loop)
print("💡 预期：知识库中没有深圳天气，可能触发 Action Loop")
print("\n" + "="*60)

answer, history, success = react_executor(question_loop, max_iterations=5, verbose=True)

print("\n" + "="*60)
print("📊 实验结果")
print("="*60)
print(f"✅ 成功: {success}")
print(f"🔢 迭代次数: {len(history)}")

# 检测 Action Loop
print("\n🔍 Action Loop 检测:")
actions = []
for step in history:
    action_match = re.search(r"Action:\s*(\w+)", step['output'])
    if action_match:
        actions.append(action_match.group(1))

print(f"Action 序列: {actions}")

# 检测连续重复
loop_detected = False
for i in range(len(actions) - 1):
    if actions[i] == actions[i+1]:
        print(f"⚠️ 检测到连续重复 Action: {actions[i]}")
        loop_detected = True
        break

if loop_detected:
    print("❌ 确认：发生 Action Loop 失败模式")
else:
    print("✅ 未发生 Action Loop")

## ❌ 实验 3：失败模式 2 - Thought 碎片化

**问题描述**：模型的思考过程过于简短或不连贯，失去推理链

**典型表现**：
```
Thought: 搜索  ← 太简短！
Action: Search
Action Input: Python

Thought: 好  ← 无意义！
Action: Calculator
...
```

**vs 正常 Thought**：
```
Thought: 要回答这个问题，我需要先了解埃菲尔铁塔的高度，然后与东方明珠的468米进行比较
Action: Search
Action Input: 埃菲尔铁塔
```

**影响**：
- 推理链条断裂
- 难以 debug
- 容易出错

**解决方案**：
- 在 Prompt 中强调详细思考
- 提供 Few-shot 示例
- 增加 Thought 长度要求

In [ ]:
# 实验 3：Thought 碎片化检测
# 使用之前成功案例的历史记录进行分析

def analyze_thought_quality(history):
    """分析 Thought 质量"""
    thoughts = []
    
    for step in history:
        thought_match = re.search(r"Thought:\s*(.+?)(?=Action:|Final Answer:|$)", step['output'], re.DOTALL)
        if thought_match:
            thought = thought_match.group(1).strip()
            thoughts.append(thought)
    
    print(f"📊 发现 {len(thoughts)} 个 Thought")
    print("\n" + "="*60)
    
    fragmented_count = 0
    for i, thought in enumerate(thoughts, 1):
        length = len(thought)
        word_count = len(thought.split())
        
        # 碎片化判断标准
        is_fragmented = length < 10 or word_count < 3
        
        status = "❌ 碎片化" if is_fragmented else "✅ 正常"
        print(f"\nThought {i}: {status}")
        print(f"  内容: {thought[:50]}{'...' if len(thought) > 50 else ''}")
        print(f"  长度: {length} 字符, {word_count} 词")
        
        if is_fragmented:
            fragmented_count += 1
    
    print("\n" + "="*60)
    print(f"📈 碎片化率: {fragmented_count}/{len(thoughts)} ({fragmented_count/len(thoughts)*100:.1f}%)")
    
    if fragmented_count > len(thoughts) * 0.3:  # 超过30%碎片化
        print("⚠️ 警告：Thought 碎片化严重！")
    else:
        print("✅ Thought 质量良好")
    
    return fragmented_count, len(thoughts)

# 使用之前的成功案例分析
if 'history' in locals() and history:
    print("🔍 分析上一个实验的 Thought 质量:")
    analyze_thought_quality(history)
else:
    print("⚠️ 没有可用的历史记录，跳过分析")

## ❌ 实验 4：失败模式 3 - 工具幻觉（Tool Hallucination）

**问题描述**：模型调用不存在的工具，或错误使用工具

**典型表现**：
```
Action: GoogleSearch  ← 不存在的工具！
Action: Database      ← 没有这个工具！
Action: EmailSend     ← 幻觉出来的！
```

**实际可用工具**：
- Search
- Calculator
- Lookup

**触发原因**：
1. 训练数据中见过类似工具名
2. 工具列表描述不清晰
3. 缺少严格的工具名称约束

**危害**：
- 执行失败
- 浪费迭代次数
- 可能导致系统崩溃（如果没有错误处理）

**解决方案**：
- 明确列出可用工具
- 添加工具名称验证
- 返回清晰的错误信息

In [ ]:
# 实验 4：工具幻觉检测

def detect_tool_hallucination(history, valid_tools):
    """检测工具幻觉"""
    print(f"🔧 有效工具列表: {list(valid_tools.keys())}")
    print("\n" + "="*60)
    
    hallucinations = []
    all_actions = []
    
    for i, step in enumerate(history, 1):
        action_match = re.search(r"Action:\s*(\w+)", step['output'])
        if action_match:
            action = action_match.group(1)
            all_actions.append(action)
            
            if action not in valid_tools:
                hallucinations.append({
                    'step': i,
                    'action': action,
                    'output': step['output']
                })
                print(f"\n❌ 步骤 {i}: 检测到工具幻觉")
                print(f"   幻觉工具: {action}")
                print(f"   相似工具: {_find_similar_tool(action, valid_tools)}")
    
    print("\n" + "="*60)
    print(f"📊 统计:")
    print(f"  总 Action 数: {len(all_actions)}")
    print(f"  幻觉数: {len(hallucinations)}")
    
    if hallucinations:
        print(f"  幻觉率: {len(hallucinations)/len(all_actions)*100:.1f}%")
        print("\n⚠️ 发现工具幻觉问题！")
    else:
        print("  幻觉率: 0%")
        print("\n✅ 未发现工具幻觉")
    
    return hallucinations

def _find_similar_tool(hallucinated_tool, valid_tools):
    """寻找相似的有效工具"""
    hallucinated_lower = hallucinated_tool.lower()
    for valid_tool in valid_tools.keys():
        if hallucinated_lower in valid_tool.lower() or valid_tool.lower() in hallucinated_lower:
            return valid_tool
    return "无相似工具"

# 分析之前实验的工具使用
if 'history' in locals() and history:
    print("🔍 检测工具幻觉:")
    detect_tool_hallucination(history, TOOLS)
else:
    # 演示工具幻觉的例子
    print("📝 工具幻觉示例:")
    print("\n常见幻觉工具:")
    fake_tools = ["GoogleSearch", "WebBrowser", "Database", "Email", "FileRead"]
    for tool in fake_tools:
        print(f"  ❌ {tool} (不存在)")
    
    print(f"\n✅ 实际可用: {list(TOOLS.keys())}")

## ❌ 实验 5：失败模式 4 - 过早终止（Premature Stop）

**问题描述**：模型在问题未解决前就给出 Final Answer

**典型表现**：
```
Thought: 我需要搜索巴黎的信息
Action: Search
Action Input: 巴黎
Observation: 巴黎是法国首都，人口约220万

Thought: 我现在知道最终答案了  ← 还没解决问题！
Final Answer: 巴黎是法国首都     ← 答非所问！
```

**vs 正确流程**：
```
问题: 巴黎的埃菲尔铁塔有多高？

1. Search[巴黎] → 获取基本信息
2. Search[埃菲尔铁塔] → 获取高度信息  ← 需要继续！
3. Final Answer: 324米
```

**触发原因**：
1. 模型误以为已经找到答案
2. 缺少验证步骤
3. Prompt 没有明确"必须解决原始问题"

**检测方法**：
- 检查 Final Answer 是否真正回答了问题
- 统计 Action 次数（过少可能是过早终止）

In [ ]:
# 实验 5：过早终止检测

def detect_premature_stop(question, answer, history, min_actions=2):
    """检测过早终止"""
    print(f"🔍 原始问题: {question}")
    print(f"💡 给出答案: {answer}")
    print("\n" + "="*60)
    
    # 统计 Action 数量
    action_count = sum(1 for step in history if "Action:" in step['output'])
    print(f"📊 执行了 {action_count} 次 Action")
    
    # 检测 1: Action 次数过少
    if action_count < min_actions:
        print(f"⚠️ 警告: Action 次数少于 {min_actions}，可能过早终止")
        premature = True
    else:
        print(f"✅ Action 次数充足")
        premature = False
    
    # 检测 2: 分析问题复杂度
    question_keywords = ["多少", "计算", "比较", "差", "和"]
    needs_calculation = any(kw in question for kw in question_keywords)
    
    if needs_calculation:
        print(f"\n🧮 问题需要计算")
        # 检查是否使用了 Calculator
        used_calculator = any("Calculator" in step['output'] for step in history)
        if not used_calculator:
            print("⚠️ 警告: 需要计算但未使用 Calculator，可能答案不完整")
            premature = True
        else:
            print("✅ 已使用 Calculator")
    
    # 检测 3: 关键词匹配
    print(f"\n🔍 答案相关性检测:")
    # 简化检测：检查答案长度
    if answer and len(answer) < 5:
        print(f"⚠️ 警告: 答案过短 ({len(answer)} 字符)，可能不完整")
        premature = True
    else:
        print(f"✅ 答案长度合理 ({len(answer) if answer else 0} 字符)")
    
    print("\n" + "="*60)
    if premature:
        print("❌ 检测结果: 可能发生过早终止")
    else:
        print("✅ 检测结果: 未发现过早终止")
    
    return premature

# 使用一个容易过早终止的问题测试
question_premature = "Python 语言的创建者是谁？他什么时候创建的？"

print("🧪 测试过早终止场景")
print("="*60)

answer, history, success = react_executor(question_premature, max_iterations=3, verbose=False)

if success:
    print(f"\n最终答案: {answer}")
    print("\n开始检测:")
    detect_premature_stop(question_premature, answer, history, min_actions=2)
else:
    print("❌ 任务失败，无法检测")

## 📊 实验总结

### ✅ ReAct 成功要素

1. **清晰的格式定义**
   - 明确 Thought-Action-Observation 循环
   - 提供 Final Answer 终止条件

2. **合理的迭代限制**
   - 设置 `max_iterations` 防止无限循环
   - 一般 3-5 次足够简单任务

3. **详细的 Thought**
   - 避免碎片化（长度 > 10 字符）
   - 体现推理链条

4. **工具验证**
   - 严格检查工具名称
   - 返回清晰错误信息

### ❌ 四大失败模式对比

| 失败模式 | 表现 | 检测方法 | 解决方案 |
|---------|------|---------|----------|
| **Action Loop** | 重复调用相同工具 | 检测连续重复 Action | 限制迭代次数 |
| **Thought 碎片化** | 思考过于简短 | 统计 Thought 长度 | Prompt 要求详细思考 |
| **工具幻觉** | 调用不存在的工具 | 验证工具名称 | 明确列出可用工具 |
| **过早终止** | 未解决问题就结束 | 检查 Action 数量和答案相关性 | 强调"完整回答原问题" |

### 🎯 最佳实践

```python
# 1. 清晰的系统提示
system_prompt = """
工具列表：[明确列出]
格式要求：[严格定义]
终止条件：[Final Answer]
"""

# 2. 迭代限制
max_iterations = 5  # 防止 Action Loop

# 3. 工具验证
if action not in TOOLS:
    return f"错误：未知工具 '{action}'"

# 4. 详细日志
verbose = True  # 便于 debug
```

### 🔄 ReAct vs CoT

| 维度 | CoT | ReAct |
|-----|-----|-------|
| 能力范围 | 纯推理 | 推理 + 工具调用 |
| 外部信息 | ❌ | ✅ 通过工具获取 |
| 实时性 | ❌ | ✅ 可查询最新数据 |
| 复杂度 | 低 | 高（需要工具管理） |
| 主要失败 | 推理错误 | Loop、幻觉、碎片化 |

### 💡 应用场景

- **适合 ReAct**: 需要查询、计算、多步骤的任务
- **适合 CoT**: 纯推理、数学证明、逻辑分析

**ReAct 是现代 AI Agent 的核心模式！**